In [10]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
form_geo_df  = gpd.read_file(r"D:\Mestrado\Trabalho Final\SIG\Unidades Geológicas.zip")

In [ ]:
columns = set()

for val in [
    "aluv", "detrit", "coluv", "calc", "dolom", "marga", "arenit", "quartzaren",
    "conglom", "argil", "pelit", "silt", "folhelho", "quartzit", "granito",
    "granodior", "sienito", "riolito", "dacito", "gnaiss", "migmat", "ultramaf",
    "itabir", "ferrit", "ferrugin"]:
    for column in form_geo_df.columns:
        try:
            t_f = form_geo_df[column].str.lower().str.contains(val).any()
            if(t_f):
                columns.add(column)
        except:
            pass

columns

{'dominio',
 'hierarquia',
 'litotipo1',
 'litotipo2',
 'nome_unida',
 'textura',
 'unigeo'}

# Classificação com base nos dados das unidades geológicas

- Litotipo 1
- Litotipo 2
- Classe da Rocha

In [43]:
form_geo_df.gr_coer.unique()

array(['VariÃ¡vel na horizontal e vertical', 'Duras',
       'VariÃ¡vel na vertical', 'MÃ©dias', 'Muito brandas', 'Brandas'],
      dtype=object)

In [46]:
litotipo1 = form_geo_df["litotipo1"].str.lower()
poro_pri  = form_geo_df['poro_pri'].str.lower()
cis_frat  = form_geo_df['cis_frat'].str.lower()
intemp_f  = form_geo_df['intemp_f'].str.lower()
intemp_q  = form_geo_df['intemp_q'].str.lower()
textura   = form_geo_df['textura'].str.lower()
gr_coer   = form_geo_df['gr_coer'].str.lower()

# Normalização
t1 = None
for i in ['dominio', 'hierarquia', 'litotipo1', 'litotipo2', 'nome_unida', 'textura', 'unigeo']:
    # mask_none = np.isnan(form_geo_df[i].values)
    if t1 is None:
        t1 = form_geo_df[i].astype(str).str.lower()
    else:
        t1 += form_geo_df[i].astype(str).str.lower()

In [69]:
# Litologia genérica ou não reconhecida → classificação intermediária
form_geo_df["class_infil"] = 5

# Depósitos aluvionares → alta permeabilidade por porosidade primária elevada
mask = t1.str.contains("aluv").index
form_geo_df.loc[mask, "class_infil"] = 9

# Rochas carbonáticas → fraturamento + carstificação favorecem infiltração
mask = t1.str.contains("calc") | t1.str.contains("dolom") | t1.str.contains("marga")
form_geo_df.loc[mask, "class_infil"] = 8

# Coberturas detrito-coluvionares → materiais soltos, boa infiltração
mask = t1.str.contains("detrit") | t1.str.contains("coluv")
form_geo_df.loc[mask, "class_infil"] = 7

# Arenitos → porosidade intergranular significativa
mask = t1.str.contains("arenit") | t1.str.contains("quartzaren")
form_geo_df.loc[mask, "class_infil"] = 7

# Conglomerados → porosidade moderada com fluxo entre clastos
mask = t1.str.contains("conglom")
form_geo_df.loc[mask, "class_infil"] = 6

# Pelitos → porosidade baixa, alta retenção de água → baixa infiltração
mask = t1.str.contains("argil") | t1.str.contains("pelit") | t1.str.contains("silt") | t1.str.contains("folhelho")
form_geo_df.loc[mask, "class_infil"] = 4

# Quartzitos → muito compactos, baixa porosidade e baixa infiltração
mask = t1.str.contains("quartzit")
form_geo_df.loc[mask, "class_infil"] = 3

# Ígneas félsicas → porosidade primária quase zero; infiltração depende só de fraturas
mask = t1.str.contains("granito") | t1.str.contains("granodior") | t1.str.contains("sienito") | t1.str.contains("riolito") | t1.str.contains("dacito")
form_geo_df.loc[mask, "class_infil"] = 3

# Gnaisses/migmatitos → extremamente compactos, baixa porosidade
mask = t1.str.contains("gnaiss") | t1.str.contains("migmat")
form_geo_df.loc[mask, "class_infil"] = 2

# Ultramáficas/itabiritos → muito densas, baixíssima infiltração
mask = t1.str.contains("ultramaf") | t1.str.contains("itabir") | t1.str.contains("ferrit") | t1.str.contains("ferrugin")
form_geo_df.loc[mask, "class_infil"] = 1

form_geo_df["class_infil"]

0       1
1       3
2       3
3       3
4       1
       ..
5671    3
5672    3
5673    3
5674    3
5675    3
Name: class_infil, Length: 5676, dtype: int64

In [70]:
# Modificadores positivos

## Porosidade

# Alta porosidade primária → incremento direto da infiltração
mask = poro_pri.str.contains("alta")
form_geo_df.loc[mask, "class_infil"] += 2
mask_ans = mask

# Porosidade média → melhora parcial
mask = poro_pri.str.contains("moderada")
form_geo_df.loc[(mask_ans & mask), "class_infil"] -= 2
form_geo_df.loc[mask, "class_infil"] += 2

## Fraturamento

# Fraturamento elevado → facilita infiltração em rochas densas
mask = cis_frat.str.contains("intensamente")
form_geo_df.loc[mask, "class_infil"] += 1

## Intemperismos

# Intemperismo físico → aumenta porosidade secundária
mask = intemp_f.str.contains("alta")
form_geo_df.loc[mask, "class_infil"] += 1

# Intemperismo químico → aumenta dissolução e condutividade do meio
mask = intemp_q.str.contains("alta")
form_geo_df.loc[mask, "class_infil"] += 1

form_geo_df["class_infil"]

0       3
1       5
2       5
3       5
4       3
       ..
5671    3
5672    3
5673    3
5674    3
5675    3
Name: class_infil, Length: 5676, dtype: int64

In [72]:
# Modificadores negativos

## Tipo da rocha

# Rocha muito coerente → compactação → menor infiltração
mask = gr_coer.str.contains("duras")
form_geo_df.loc[mask, "class_infil"] -= 1

## Textura

# Argilosa → microporos → baixa condutividade
mask = textura.astype(str).str.contains("argil")
form_geo_df.loc[mask, "class_infil"] -= 2
mask_ans = mask

# Siltosa → infiltração limitada
mask = textura.astype(str).str.contains("silt")
form_geo_df.loc[(mask_ans & mask), "class_infil"] += 2
form_geo_df.loc[mask, "class_infil"] -= 1

form_geo_df["class_infil"]

0       2
1       1
2       3
3       1
4       3
       ..
5671   -1
5672   -1
5673   -1
5674   -1
5675   -1
Name: class_infil, Length: 5676, dtype: int64

In [76]:
# Normalizando
print("Min", form_geo_df["class_infil"].min(), "Max", form_geo_df["class_infil"].max())
form_geo_df["class_infil"] = form_geo_df["class_infil"].apply(lambda x: 1 if x < 1 else 9 if x > 9 else x)
print("Min", form_geo_df["class_infil"].min(), "Max", form_geo_df["class_infil"].max())

Min 1 Max 9
Min 1 Max 9


In [77]:
# Salvando DF

form_geo_df.to_file("FormGeo.shp")

C:\Users\felip\AppData\Local\Temp\ipykernel_29828\3448364656.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  form_geo_df.to_file("FormGeo.shp")
d:\Mestrado\Trabalho Final\Codigos\libs\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'class_infil' to 'class_infi'
  ogr_write(
